# 14. Movielens 영화 추천

MF 모델 학습 방법을 토대로 영화 추천 시스템을 제작해보자. 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens이다.   

유저가 영화에 대해 평점을 매긴 데이터가 크기별로 있는데 이번에는 `MovieLens 1M Dataset`을 사용한다. 별점 데이터는 대표적인 명시적(explicit) 평가 데이터인데, 암묵적(implicit) 평가 데이터로 간주하고 테스트해보자. 어떻게 하냐면 __별점을 시청횟수__로 해석하자. __3점 미만__으로 준 데이터는 __선호하지 않는다__고 가정하자.

# 1. 데이터 준비와 전처리

### 1-1. 데이터셋 준비하기

```
$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m
```

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
ratings.shape

(1000209, 4)

### 1-2. 전처리하기
별점을 시청횟수로 보고 3점 미만은 선호하지 않는다고 가정했기 때문에, 3점 이상만 남기고 ratings를 counts로 바꾸자.


In [3]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 이름을 counts로 바꾼다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings.shape

(836478, 4)

In [6]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies.shape

(3883, 3)

# 2. 데이터 분석하기

### 2-1. 영화 개수 파악하기

In [9]:
ratings['movie_id'].nunique()

3628

In [10]:
movies['movie_id'].nunique()

3883

> 별점을 준 영화의 총 개수와 movies 데이터 불러온 영화 개수가 상이하다. 전처리 과정에서 유실됐나 싶어 확인을 했는데 그건 아니다.

### 2-2. 사용자 수 파악하기

In [11]:
ratings['user_id'].nunique()

6039

### 2-3. 가장 인기 있는 영화 30개 알아보기

In [12]:
test_count = ratings.groupby('movie_id')['user_id'].count()
test_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [13]:
# 데이터프레임 합치기 두 데이터프레임의 동일한 feautres인 movie_id를 기준으로 합치게 된다.
ratings_movies = pd.merge(ratings, movies)
ratings_movies

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [14]:
movies_count = ratings_movies.groupby('title')['user_id'].count()
best_movies = movies_count.sort_values(ascending=False).head(30)
best_movies

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 3. 선호하는 영화를 골라 사용자 정보 세팅하기

위의 best_movies를 보고 내가 좋아하는 영화는 대중적인 영화임을 알 수 있었다. 이들을 추가하자.

In [15]:
my_favorite = ['Terminator 2: Judgment Day', 'Back to the Future (1985)', 'Matrix, The (1999)', 
               'Men in Black (1997)', 'Toy Story (1995)']
my_farvorite_movie_id = [589, 1270, 2571, 1580, 1]

my_playlist = pd.DataFrame({'user_id': ['hou'] * 5, 'title': my_favorite, 
                            'movie_id': my_farvorite_movie_id, 'counts': [5]*5})

if not ratings_movies.isin({'user_id':['hou']})['user_id'].any():  # user_id에 'hou'이라는 데이터가 없다면
    ratings_movies = ratings_movies.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해준다. 

ratings_movies.tail(10)       # 잘 추가되었는지 확인하기

,user_id,movie_id,counts,timestamp,title,genre
836473,5851,3607,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,hou,589,5,NaN,Terminator 2: Judgment Day,NaN
1,hou,1270,5,NaN,Back to the Future (1985),NaN
2,hou,2571,5,NaN,"Matrix, The (1999)",NaN
3,hou,1580,5,NaN,Men in Black (1997),NaN
4,hou,1,5,NaN,Toy Story (1995),NaN


In [16]:
# 사용하는 컬럼만 남겨주자
using_cols = ['user_id', 'movie_id', 'counts']
data = ratings_movies[using_cols]
data.tail()

,user_id,movie_id,counts
0,hou,589,5
1,hou,1270,5
2,hou,2571,5
3,hou,1580,5
4,hou,1,5


### 3-1. indexing

In [17]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [18]:
# 인덱싱 확인
print(user_to_idx['hou'])    # 6039명의 유저 중 마지막으로 추가된 유저이니 인덱스값 6039이 나와야 한다.
print(movie_to_idx[2])

6039
513


In [19]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다.
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!


/tmp/ipykernel_357/2264975106.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다.


movie column indexing OK!!


/tmp/ipykernel_357/2264975106.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['movie_id'] = temp_movie_data


,user_id,movie_id,counts
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,92,5
1,6039,22,5
2,6039,124,5
3,6039,175,5


# 4. CSR matrix 만들기

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. 학습하기

In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분이다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

`AlternatingLeastSquares` 클래스의 `__init__` 파라미터를 살펴보면 다음과 같다.   
- factors: 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization: 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu: gpu 사용할 것인지
- iterations: epochs와 같은 의미로, 데이터를 몇 번 반복해서 학습할 것인지   

factors와 iterations를 늘릴수록 잘 학습하지만 over fitting의 우려가 있다.

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해준다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

# 6. 결과 확인하기

### 6-1. 내가 선호하는 5가지 영화 중 하나를 골라 모델이 예측한 나의 선호도를 파악해보자.

모델이 나와 `Back to the Future (1985)`의 선호도를 예측해보자.

In [25]:
hou, back_to_the_future = user_to_idx['hou'], movie_to_idx[1270]
hou_vector, back_to_the_future_vector = als_model.user_factors[hou], als_model.item_factors[back_to_the_future]

In [26]:
# hou_vector와 back_to_the_future_vector를 내적하는 코드
np.dot(hou_vector, back_to_the_future_vector)

0.81237566

0.79가 나왔는데 이 결과만 갖고 모델의 선호 기준을 정하기 어렵다. 왜냐하면 모델 만드는 사람이 정하기 나름이기 때문이다. 객관적 지표로 만들어 기준을 정하거나, 도메인 경험을 통해 정할 수도 있다. 추천 시스템은 다른 머신러닝 task보다 객관적인 평가가 어려운 분야이다.

### 6-2. 내가 좋아하는 영화와 비슷한 영화를 추천받아보자.

In [27]:
movie_id = movie_to_idx[1270] # Back to the Future (1985)
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(22, 0.99999994),
 (674, 0.42662445),
 (13, 0.42095563),
 (488, 0.28924245),
 (243, 0.28662732),
 (615, 0.2821662),
 (19, 0.2693895),
 (162, 0.2420819),
 (91, 0.23768935),
 (1144, 0.23670787),
 (163, 0.22797248),
 (2765, 0.21829861),
 (675, 0.21639761),
 (680, 0.21346776),
 (44, 0.21195726)]

In [28]:
# 사용하는 컬럼만 남겨주자
using_cols = ['movie_id', 'title']
data = ratings_movies[using_cols]
data.tail()

,movie_id,title
0,589,Terminator 2: Judgment Day
1,1270,Back to the Future (1985)
2,2571,"Matrix, The (1999)"
3,1580,Men in Black (1997)
4,1,Toy Story (1995)


In [29]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict 생성하기
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1270,
 2011,
 2918,
 1307,
 2716,
 2407,
 2797,
 3421,
 3471,
 3361,
 1394,
 3171,
 2012,
 541,
 260]

In [30]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [31]:
targets = get_similar_movie(1270)
get_similar_movie(1270)

[1270, 2011, 2918, 1307, 2716, 2407, 2797, 3421, 3471, 3361]

In [32]:
similar_df = data.loc[data['movie_id'].isin(targets)]
print(similar_df.drop_duplicates())

        movie_id                                      title
13595       2918            Ferris Bueller's Day Off (1986)
18921       2797                                 Big (1988)
21777       1270                  Back to the Future (1985)
88041       3471  Close Encounters of the Third Kind (1977)
155567      3421                        Animal House (1978)
217209      2716                        Ghostbusters (1984)
350263      1307             When Harry Met Sally... (1989)
414869      2407                              Cocoon (1985)
445284      2011          Back to the Future Part II (1989)
619075      3361                         Bull Durham (1988)


### 6-3. 내가 좋아할 만한 영화를 추천받아보자.

In [33]:
user = user_to_idx['hou']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(107, 0.48013276),
 (62, 0.38939273),
 (200, 0.3758145),
 (50, 0.30811006),
 (110, 0.267884),
 (141, 0.25710303),
 (145, 0.21835846),
 (236, 0.2113647),
 (375, 0.20785072),
 (317, 0.174451),
 (117, 0.17097746),
 (322, 0.16979437),
 (288, 0.16929853),
 (60, 0.16879389),
 (674, 0.16076647),
 (372, 0.15928304),
 (44, 0.15304127),
 (479, 0.14737636),
 (150, 0.14539132),
 (87, 0.14486977)]

In [34]:
recommend_df = [idx_to_movie[i[0]] for i in movie_recommended]
recommend_df

[480,
 2916,
 1240,
 3114,
 1265,
 457,
 1527,
 377,
 1573,
 32,
 1196,
 34,
 1517,
 2628,
 2011,
 3793,
 260,
 1584,
 780,
 110]

In [35]:
recommend_df = data.loc[data['movie_id'].isin(targets)]
print(recommend_df.drop_duplicates())

        movie_id                                      title
13595       2918            Ferris Bueller's Day Off (1986)
18921       2797                                 Big (1988)
21777       1270                  Back to the Future (1985)
88041       3471  Close Encounters of the Third Kind (1977)
155567      3421                        Animal House (1978)
217209      2716                        Ghostbusters (1984)
350263      1307             When Harry Met Sally... (1989)
414869      2407                              Cocoon (1985)
445284      2011          Back to the Future Part II (1989)
619075      3361                         Bull Durham (1988)


# 회고

이번 추천 시스템 프로젝트는 아이디어가 많이 떠올라 매우 재미있었고 동시에 아쉬운점도 많이 있었다.   

먼저 익스 이름부터 아이유팬이 좋아할만한 다른 아티스트 찾기였다. 국내 기준으로 스터디를 진행했으면 결과를 보면서 공감이 더 많이 됐을거같은데, 외국 기준으로 해서 모르는 아티스트가 많아 아쉬웠다. 그리고 외국 영화도 많이 알지 못해서 내가 아는 국내 음악이나, 국내 영화로 데이터를 만졌으면 결과를 확인할 때 더 직관적으로 와닿아서 좋았을거같다. 

모델의 파라미터(`factors, iterations`)를 많이 올렸는데, 그래도 결과값들이 많이 적게 나왔다. 더 올리면 overfitting이 나올거같아 크게 올리지 않고 진행했는데, 여기 관련해서 더 공부하면 좋을거같다.

이번에 영화 별점을 시청회수로 바꾸고 진행했는데, 정말로 암묵적인 데이터를 갖고 다뤄보고 싶다. 그리고 이번처럼 명시적인 데이터로 진행하는 경우에 다른 요소들도 더 많이 추가했으면 좋았을거같은데 아쉽다.

movie_id를 보고 title과 바로 교체해주는 방법을 구상하였는데, 구현하기 어려웠다. 그래서 내가 사용한 방법은 데이터프레임끼리 합치거나 마지막 추천받는 두 가지는 영화 데이터프레임에서 movie_id가 같지 않으면 삭제하고 서로 데이터가 겹쳐도 삭제하는 방식으로 진행했다. 하지만 개인적으로 마음에 들지 않고 다른 방법을 찾고싶다.